![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/streamlit_notebooks/healthcare_jsl/ER_SNOMED_ES.ipynb)

# Colab Setup

In [ ]:
# Install the johnsnowlabs library to access Spark-OCR and Spark-NLP for Healthcare, Finance, and Legal.
! pip install -q johnsnowlabs

In [ ]:
from google.colab import files
print("Please Upload your John Snow Labs License using the button below")
license_keys = files.upload()

In [ ]:
from johnsnowlabs import *

# After uploading your license run this to install all licensed Python Wheels and pre-download Jars the Spark Session JVM
# Make sure to restart your notebook afterwards for changes to take effect

jsl.install()

# Start Session

In [ ]:
from johnsnowlabs import *
# Automatically load license data and start a session with all jars user has access to
spark = jsl.start()

# Checking in an inference pipeline

In [ ]:
documentAssembler = nlp.DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

sentenceDetector = nlp.SentenceDetectorDLModel.pretrained() \
    .setInputCols(["document"]) \
    .setOutputCol("sentence")

tokenizer = nlp.Tokenizer()\
    .setInputCols("sentence")\
    .setOutputCol("token")

word_embeddings = nlp.RoBertaEmbeddings.pretrained("roberta_base_biomedical", "es")\
    .setInputCols(["document", "token"])\
    .setOutputCol("roberta_embeddings")

ner = medical.NerModel.pretrained("roberta_ner_diag_proc","es","clinical/models")\
    .setInputCols("sentence","token","roberta_embeddings")\
    .setOutputCol("ner")

ner_converter = nlp.NerConverter() \
    .setInputCols(["sentence", "token", "ner"]) \
    .setOutputCol("ner_chunk")

snomed_prepro_pipeline = Pipeline(
    stages = [
        documentAssembler,
        sentenceDetector,
        tokenizer,
        word_embeddings,
        ner,
        ner_converter
    ])

empty = spark.createDataFrame([['']]).toDF("text")

p_model = snomed_prepro_pipeline.fit(empty)

sentence_detector_dl download started this may take some time.
Approximate size to download 354.6 KB
[OK!]
roberta_base_biomedical download started this may take some time.
Approximate size to download 288 MB
[OK!]
roberta_ner_diag_proc download started this may take some time.
[OK!]


In [ ]:
import pandas as pd

test_sentence = 'Mujer de 28 años con antecedentes de diabetes mellitus gestacional diagnosticada ocho años antes de la presentación y posterior diabetes mellitus tipo dos (DM2), un episodio previo de pancreatitis inducida por HTG tres años antes de la presentación, asociado con una hepatitis aguda, y obesidad con un índice de masa corporal (IMC) de 33,5 kg / m2, que se presentó con antecedentes de una semana de poliuria, polidipsia, falta de apetito y vómitos. Dos semanas antes de la presentación, fue tratada con un ciclo de cinco días de amoxicilina por una infección del tracto respiratorio. Estaba tomando metformina, glipizida y dapagliflozina para la DM2 y atorvastatina y gemfibrozil para la HTG. Había estado tomando dapagliflozina durante seis meses en el momento de la presentación. El examen físico al momento de la presentación fue significativo para la mucosa oral seca; significativamente, su examen abdominal fue benigno sin dolor a la palpación, protección o rigidez. Los hallazgos de laboratorio pertinentes al ingreso fueron: glucosa sérica 111 mg / dl, bicarbonato 18 mmol / l, anión gap 20, creatinina 0,4 mg / dl, triglicéridos 508 mg / dl, colesterol total 122 mg / dl, hemoglobina glucosilada (HbA1c) 10%. y pH venoso 7,27. La lipasa sérica fue normal a 43 U / L. Los niveles séricos de acetona no pudieron evaluarse ya que las muestras de sangre se mantuvieron hemolizadas debido a una lipemia significativa. La paciente ingresó inicialmente por cetosis por inanición, ya que refirió una ingesta oral deficiente durante los tres días previos a la admisión. Sin embargo, la química sérica obtenida seis horas después de la presentación reveló que su glucosa era de 186 mg / dL, la brecha aniónica todavía estaba elevada a 21, el bicarbonato sérico era de 16 mmol / L, el nivel de triglicéridos alcanzó un máximo de 2050 mg / dL y la lipasa fue de 52 U / L. Se obtuvo el nivel de β-hidroxibutirato y se encontró que estaba elevado a 5,29 mmol / L; la muestra original se centrifugó y la capa de quilomicrones se eliminó antes del análisis debido a la interferencia de la turbidez causada por la lipemia nuevamente. El paciente fue tratado con un goteo de insulina para euDKA y HTG con una reducción de la brecha aniónica a 13 y triglicéridos a 1400 mg / dL, dentro de las 24 horas. Se pensó que su euDKA fue precipitada por su infección del tracto respiratorio en el contexto del uso del inhibidor de SGLT2. La paciente fue atendida por el servicio de endocrinología y fue dada de alta con 40 unidades de insulina glargina por la noche, 12 unidades de insulina lispro con las comidas y metformina 1000 mg dos veces al día. Se determinó que todos los inhibidores de SGLT2 deben suspenderse indefinidamente. Tuvo un seguimiento estrecho con endocrinología post alta.'
res = p_model.transform(spark.createDataFrame(pd.DataFrame({'text': [test_sentence]})))

In [ ]:
c2doc = nlp.Chunk2Doc() \
    .setInputCols("ner_chunk") \
    .setOutputCol("sentence")

chunk_tokenizer = nlp.Tokenizer()\
    .setInputCols("sentence")\
    .setOutputCol("token")

chunk_word_embeddings = nlp.RoBertaEmbeddings.pretrained("roberta_base_biomedical", "es")\
    .setInputCols(["sentence", "token"])\
    .setOutputCol("ner_chunk_word_embeddings")

chunk_embeddings = nlp.SentenceEmbeddings() \
    .setInputCols(["sentence", "ner_chunk_word_embeddings"]) \
    .setOutputCol("ner_chunk_embeddings") \
    .setPoolingStrategy("AVERAGE")

er = medical.SentenceEntityResolverModel.pretrained("robertaresolve_snomed", "es", "clinical/models")\
    .setInputCols(["sentence", "ner_chunk_embeddings"]) \
    .setOutputCol("snomed_code") \
    .setDistanceFunction("EUCLIDEAN")

snomed_resolve_pipeline = Pipeline(
    stages = [
        c2doc,
        chunk_tokenizer,
        chunk_word_embeddings,
        chunk_embeddings,
        er
        ])


roberta_base_biomedical download started this may take some time.
Approximate size to download 288 MB
[OK!]
robertaresolve_snomed download started this may take some time.
[OK!]


In [ ]:
p_infer_model = snomed_resolve_pipeline.fit(res.select('ner_chunk'))

In [ ]:
final_res = p_infer_model.transform(res.select('ner_chunk'))

In [ ]:
import pandas as pd

chunk = 'ner_chunk'
output_col='snomed_code'

pd.set_option('display.max_colwidth', 0)

df = final_res.select(F.explode(F.arrays_zip(final_res[chunk].begin, 
                                             final_res[chunk].end, 
                                             final_res[chunk].result, 
                                             final_res[chunk].metadata, 
                                             final_res[output_col].result, 
                                             final_res[output_col].metadata)).alias("cols")) \
              .select(F.expr("cols['0']").alias("begin"),
                      F.expr("cols['1']").alias("end"),
                      F.expr("cols['3']['sentence']").alias("sent_id"),
                      F.expr("cols['2']").alias("ner_chunk"),
                      F.expr("cols['3']['entity']").alias("entity"), 
                      F.expr("cols['4']").alias("snomed_code"),
                      F.expr("cols['5']['all_k_resolutions']").alias("snomed_texts"),                                     
                      ).toPandas()

In [ ]:
df

,begin,end,sent_id,ner_chunk,entity,snomed_code,snomed_texts
0,37,65,0,diabetes mellitus gestacional,DIAGNOSTICO,11687002,diabetes mellitus gestacional:::diabetes mellitus gestacional:::diabetes mellitus inestable:::diabetes mellitus inestable:::diabetes mellitus materna:::diabetes mellitus medicamentosa:::diabetes mellitus postrasplante:::diabetes mellitus atípica:::diabetes mellitus gestacional posparto:::diabetes mellitus lipoatrófica:::diabetes mellitus neonatal:::diabetes mellitus neonatal permanente:::diabetes mellitus con gangrena:::diabetes mellitus gestacional no controlada:::diabetes mellitus autosómica dominante:::diabetes mellitus tipo I inestable:::diabetes mellitus con cetoacidosis:::[X]otra diabetes mellitus especificada:::diabetes mellitus con complicaciones
1,128,153,0,diabetes mellitus tipo dos,DIAGNOSTICO,46635009,diabetes mellitus tipo I:::diabetes mellitus tipo II:::diabetes mellitus insulinodependiente tipo IA:::diabetes mellitus tipo II lábil:::diabetes mellitus tipo 1 lábil:::diabetes mellitus tipo MODY:::diabetes mellitus insulinodependiente tipo IB:::coma hipoglucémico en diabetes mellitus tipo I:::diabetes mellitus tipo 2 preexistente:::diabetes mellitus tipo 2 sin complicaciones:::cetoacidosis en diabetes mellitus tipo II:::diabetes mellitus tipo II complicada:::embarazo y diabetes mellitus tipo 2:::diabetes mellitus tipo 2 con gangrena:::trastorno debido a diabetes mellitus tipo II
2,184,195,0,pancreatitis,DIAGNOSTICO,75694006,pancreatitis:::pancreatalgia:::pancreatoscopio:::pancreatina:::pancreatina:::pancreatenfraxis:::pancreatitis recurrente:::pancreatitis recidivante:::pancreatitis recidivante:::pancreatitis necrosante:::pancreatitis apopléjica:::pancreatitis indolora:::pancreaticocistoduodenostomía:::pancreatitis hereditaria:::pancreaticoduodenostomía:::pancreaticogastrostomía:::pancreatitis metabólica:::pancreaticocistoyeyunostomía:::pancreatectomía:::pancreatitis aguda:::pancreatitis aguda
3,267,282,0,"hepatitis aguda,",DIAGNOSTICO,37871000,hepatitis aguda:::hepatitis C aguda (trastorno):::conjuntivitis aguda:::hepatitis vírica aguda:::hepatitis E aguda (trastorno):::hepatitis vírica aguda tipo B:::hepatitis B aguda con hepatitis D:::hepatitis focal aguda (trastorno):::linfangitis aguda del pie:::hepatitis tóxica aguda:::fiebre Q aguda (trastorno):::artritis aguda (trastorno):::leucemia aguda (categoría):::hepatitis viral aguda tipo A (trastorno):::insuficiencia aguda (hallazgo)
4,286,293,0,obesidad,DIAGNOSTICO,5476005,obesidad:::obesidad:::carcinoma:::carcinoma:::infarto:::activo:::bilis:::bilis:::celulosa:::celulosa:::ARN:::dieta:::hombre:::hombre:::adrenalina:::adrenalina:::adrenalina:::adrenalina:::adrenalina:::adrenalina+inotrópicos:::distrofia
5,302,324,0,índice de masa corporal,DIAGNOSTICO,162859006,"índice de masa corporal:::índice de masa corporal:::índice de masa corporal normal:::índice de masa corporal aumentado:::índice de masa corporal disminuido:::índice de masa corporal - hallazgo:::centilo del índice de masa corporal:::índice de masa corporal, alto K/M2:::índice de masa corporal 25-29, sobrepeso:::cálculo del índice de masa corporal:::índice de masa corporal, bajo K/M2:::índice de masa corporal, normal K/M2:::índice de masa corporal 30+, obesidad:::índice de masa corporal de 30,00 a 34,99:::índice de masa (propiedad):::analizador de índice de masa corporal bioeléctrico:::índice de masa corporal de 35,00 a 39,99:::índice de masa corporal, 40+ - obesidad grave:::índice de masa corporal inferior a 20"
6,327,329,0,IMC,DIAGNOSTICO,60621009,IMC:::mL:::mg/comida:::mg/kg/día:::mg/m2:::mg/alimento:::mg/día/kg:::mg/kg/h:::mg/dl:::mg/día:::mg/m2/día:::mg/m2/h:::mg/ml:::mg/intercambio:::mg/l:::mg/disparo:::masa:::masa:::masa:::masa
7,399,406,0,poliuria,DIAGNOSTICO,56574000,poliuria:::poliuria:::polifagia:::polifagia:::disuria:::frecuencia miccional y poliuria:::proteinuria:::rectorragia:::rectorragia:::proteinuria prerrenal:::proteinuria colicuativa:::piuria:::hematuria macroscópica:::h